The code below is very slightly modified from https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [11]:
# dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# let's make a classification tree from scratch.  Only once.  never again.

#   read some data.  let's use banknote authentication data from CMU: 
##  http://archive.ics.uci.edu/ml/datasets/banknote+authentication
df = pd.read_csv('../Data/data_banknote_authentication.csv', header = None)
df.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [5]:
# ok, now we need a GINI index.  This tells us information about how mixed up the classes are 
#  with respect to the categories we pick.  we will need to calculate a proportion for each
#  subgroup of rows to calculate the score.

# The GINI gives us a weighted score to evaluate the relationship 
#  between possible cuts.  The best score is 0, a perfect split between classes.
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
  # count all samples at split point
  n_instances = float(sum([len(group) for group in groups]))
  # sum weighted Gini index for each group
  gini = 0.0
  for group in groups:
    size = float(len(group))
    # avoid divide by zero
    if size == 0:
      continue
    score = 0.0
    # score the group based on the score for each class
    for class_val in classes:
      p = [row[-1] for row in group].count(class_val) / size
      score += p * p
    # weight the group score by its relative size
    gini += (1.0 - score) * (size / n_instances)
  return gini

In [6]:
# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


In [7]:
# Now we can score the attributes, we need to be able to split the data.
#  We will treat a split as an object containing an attribute and a split value.
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
  left, right = list(), list()
  for row in dataset:
    if row[index] < value:
      left.append(row)
    else:
      right.append(row)
  return left, right

In [8]:
# next we need to look at the dataset and determine where is the best place 
#  to make a split.  We search all rows and all attributes to find the best 
#  place to split.  When we find th best split, we return a dictionary containing
#  1) the index of the chosen attribute,
#  2) the value of the atribute to split on, and 
#  3) the groups of rows to the left and right of the split
# Select the best split point for a dataset
def get_split(dataset):
  class_values = list(set(row[-1] for row in dataset))
  b_index, b_value, b_score, b_groups = 999, 999, 999, None
  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = test_split(index, row[index], dataset)
      gini = gini_index(groups, class_values)
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
  return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [10]:
# here is a small test dataset for the code so far:
dataset = [[2.771244718,1.784783929,0],
  [1.728571309,1.169761413,0],
  [3.678319846,2.81281357,0],
  [3.961043357,2.61995032,0],
  [2.999208922,2.209014212,0],
  [7.497545867,3.162953546,1],
  [9.00220326,3.339047188,1],
  [7.444542326,0.476683375,1],
  [10.12493903,3.234550982,1],
  [6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 6.642]


In [12]:
# The best split for this simple test data set is on X1 at 6.642.  
#  Now we want to split all of the banknote authenication data.  
#  To make more than one split as above, we want to expand the 
#  procedure to recurse through each of the groups.  This is called
#  _recursive binary splitting_.  After the first split, we choose 
#  one of the resulting datasets (left, for example) and perform 
#  binary splitting on it, then choose the other (right) and do binary
#  splitting there.  The pocedure stops when one of the groups has no
#  rows.  Then the other group is fully classified.

# first lets take care of the terminal node
# Create a terminal node value
def to_terminal(group):
  outcomes = [row[-1] for row in group]
  return max(set(outcomes), key=outcomes.count)

# next, here is our recursive procedure for splitting.
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
  left, right = node['groups']
  del(node['groups'])
  # check for a no split
  if not left or not right:
    node['left'] = node['right'] = to_terminal(left + right)
    return
  # check for max depth
  if depth >= max_depth:
    node['left'], node['right'] = to_terminal(left), to_terminal(right)
    return
  # process left child
  if len(left) <= min_size:
    node['left'] = to_terminal(left)
  else:
    node['left'] = get_split(left)
    split(node['left'], max_depth, min_size, depth+1)
  # process right child
  if len(right) <= min_size:
    node['right'] = to_terminal(right)
  else:
    node['right'] = get_split(right)
    split(node['right'], max_depth, min_size, depth+1)


In [13]:
# now we can run the decision tree training for the dataset by starting
#  at the root (the first split of the dataset).
# Build a decision tree
def build_tree(train, max_depth, min_size):
  root = get_split(train)
  split(root, max_depth, min_size, 1)
  return root

In [14]:
# Finally, let's have a poor man's function for printing the tree
# Print a decision tree
def print_tree(node, depth=0):
  if isinstance(node, dict):
    print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
    print_tree(node['left'], depth+1)
    print_tree(node['right'], depth+1)
  else:
    print('%s[%s]' % ((depth*' ', node)))
 

In [15]:
# Now let's try this on test data
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 1, 1)
print_tree(tree)

[X1 < 6.642]
 [0]
 [1]


In [16]:
# In our simple example the data is perfectly divided after a 
#  single split.  Now let's look at how we can predict thee class 
#  of an arbitrary row from outside the training set.
#  The predict below performs a recursive descent, testing each 
#  node on the tree until it finds the terminal which should 
#  contain the row.

# Make a prediction with a decision tree
def predict(node, row):
  if row[node['index']] < node['value']:
    if isinstance(node['left'], dict):
      return predict(node['left'], row)
    else:
      return node['left']
  else:
    if isinstance(node['right'], dict):
      return predict(node['right'], row)
    else:
      return node['right']

In [17]:
# we can test predict on the test data set to see if we can 
#  recover the correct classes for the training values.
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
 
#  predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
  prediction = predict(stump, row)
  print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


In [54]:
# Now let's set up the main caller for accessing the algorithm.
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
  tree = build_tree(train, max_depth, min_size)
  predictions = list()
  for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)
  return(predictions, tree)

In [55]:
# Finally, let's set up our tests so that we can see how the algorithm is performing.

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
  dataset_split = list()
  dataset_copy = list(dataset)
  fold_size = int(len(dataset) / n_folds)
  for i in range(n_folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
  return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  folds = cross_validation_split(dataset, n_folds)
  scores = list()
  models = list()
  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None
    predicted, model = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in fold]
    accuracy = accuracy_metric(actual, predicted)
    scores.append(accuracy)
    models.append(model)
  return scores, models
 

In [56]:
# Test CART on Bank Note dataset
from random import seed
from random import randrange

seed(1)

dataset = df.values.tolist()
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores, trees = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Scores: [96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%


In [61]:
# we can print the trees too
for i in range(len(trees)):
    print('tree %d' %i)
    print('------------')
    print_tree(tree)
    print('------------')

tree 0
------------
[X1 < 0.322]
 [X2 < 7.644]
  [X1 < -0.398]
   [X3 < 6.220]
    [X2 < 7.327]
     [1.0]
     [1.0]
    [X2 < -4.606]
     [1.0]
     [0.0]
   [X3 < 3.114]
    [X2 < 5.897]
     [1.0]
     [0.0]
    [X1 < -0.384]
     [0.0]
     [0.0]
  [X1 < -2.742]
   [X1 < -6.200]
    [1.0]
    [1.0]
   [X1 < -2.648]
    [0.0]
    [X1 < -2.648]
     [0.0]
     [0.0]
 [X3 < -4.384]
  [X1 < 4.290]
   [X1 < 0.816]
    [1.0]
    [X1 < 0.816]
     [1.0]
     [1.0]
   [0.0]
  [X1 < 1.594]
   [X3 < -2.167]
    [X2 < 7.776]
     [1.0]
     [0.0]
    [X4 < 0.147]
     [0.0]
     [0.0]
   [X1 < 2.042]
    [X1 < 2.018]
     [0.0]
     [1.0]
    [X1 < 3.431]
     [0.0]
     [0.0]
------------
tree 1
------------
[X1 < 0.322]
 [X2 < 7.644]
  [X1 < -0.398]
   [X3 < 6.220]
    [X2 < 7.327]
     [1.0]
     [1.0]
    [X2 < -4.606]
     [1.0]
     [0.0]
   [X3 < 3.114]
    [X2 < 5.897]
     [1.0]
     [0.0]
    [X1 < -0.384]
     [0.0]
     [0.0]
  [X1 < -2.742]
   [X1 < -6.200]
    [1.0]
    [1.0]
